# Assignment-1: 10 bit Palindrome Classification

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np

### Setting Seed Value

In [2]:
# Set the random seed for reproducibility
seed_value = 42
np.random.seed(seed_value)

### Loading the Dataset

**Palindrome:** 32  
**Non-Palindrome:** 992

**First 10 Columns:** 10-bits of the input  
**Last Column:** Class label (0: Non-Palindrome, 1: Palindrome)

In [3]:
# Load dataset
df = pd.read_csv("palindrome_data.csv")
df.shape

(1024, 11)

### Balancing Palindrome Data

**Palindrome:** 992  
**Non-Palindrome:** 992

In [4]:
# Separate palindrome and non-palindrome examples
np.random.seed(seed_value)  # Set seed for consistency
palindrome_examples = df[df['y'] == 1]
non_palindrome_examples = df[df['y'] == 0]

# Oversample palindrome examples to match the number of non-palindrome examples
oversampled_palindrome = palindrome_examples.sample(n=len(non_palindrome_examples), replace=True, random_state=seed_value)

# Concatenate oversampled palindrome examples with non-palindrome examples
balanced_df = pd.concat([oversampled_palindrome, non_palindrome_examples])

# Shuffle the dataset
balanced_df = balanced_df.sample(frac=1, random_state=seed_value).reset_index(drop=True)

balanced_df.shape

(1984, 11)

### Train-Test split

The 4-Fold Cross Validation is applied on the training set  
After training, weights of the fold which gives the best result is used to predict on the test set.

**Training Set:** 80%  
**Test Set:** 20%

In [5]:
# Define train-test split function
def train_test_split(data, test_size=0.2):
    num_samples = len(data)
    num_test_samples = int(test_size * num_samples)

    # Shuffle the data
    np.random.seed(seed_value)
    shuffled_indices = np.random.permutation(num_samples)
    data = data.iloc[shuffled_indices]

    # Split the data into training and testing sets
    test_data = data[:num_test_samples]
    train_data = data[num_test_samples:]

    return train_data, test_data

# Perform train-test split
train_data, test_data = train_test_split(balanced_df, test_size=0.2)

X_train = train_data.iloc[:, :balanced_df.shape[1] - 1].to_numpy()
y_train = train_data.iloc[:, -1].to_numpy().reshape(-1, 1)

X_test = test_data.iloc[:, :balanced_df.shape[1] - 1].to_numpy()
y_test = test_data.iloc[:, -1].to_numpy().reshape(-1, 1)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1588, 10), (1588, 1), (396, 10), (396, 1))

### Neural Network Training with Momentum for Binary Classification

In [6]:
# Define sigmoid activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Define derivative of sigmoid function
def sigmoid_derivative(x):
    return x * (1 - x)

# Define binary cross-entropy loss function
def binary_crossentropy(y_true, y_pred):
    epsilon = 1e-7  # Small constant to prevent numerical instability
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip predicted values to avoid extreme values
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))


# Define function to initialize weights and biases
def initialize_parameters(input_size, hidden_size):
    W1 = np.random.randn(input_size, hidden_size)
    b1 = np.zeros((1, hidden_size))
    W2 = np.random.randn(hidden_size, 1)
    b2 = np.zeros((1, 1))
    return W1, b1, W2, b2


# Define function for forward propagation
def forward_propagation(X, W1, b1, W2, b2):
    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = sigmoid(Z2)
    return Z1, A1, Z2, A2

# Define function for backpropagation
def backward_propagation(X, y, Z1, A1, Z2, A2, W2):
    m = len(X)
    dZ2 = A2 - y
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m
    dZ1 = np.dot(dZ2, W2.T) * sigmoid_derivative(A1)
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m
    return dW1, db1, dW2, db2

# Define function for training the model
def train(X_train, y_train, hidden_size, learning_rate, momentum_rate, epochs):
    input_size = X_train.shape[1]
    W1, b1, W2, b2 = initialize_parameters(input_size, hidden_size)

    # Initialize velocities for momentum update
    v_dW1 = np.zeros_like(W1)
    v_db1 = np.zeros_like(b1)
    v_dW2 = np.zeros_like(W2)
    v_db2 = np.zeros_like(b2)

    for epoch in range(epochs):
        # Forward propagation
        Z1, A1, Z2, A2 = forward_propagation(X_train, W1, b1, W2, b2)

        # Compute loss
        loss = binary_crossentropy(y_train, A2)

        # Backward propagation
        dW1, db1, dW2, db2 = backward_propagation(X_train, y_train, Z1, A1, Z2, A2, W2)

        # Update velocities
        v_dW1 = momentum_rate * v_dW1 + learning_rate * dW1
        v_db1 = momentum_rate * v_db1 + learning_rate * db1
        v_dW2 = momentum_rate * v_dW2 + learning_rate * dW2
        v_db2 = momentum_rate * v_db2 + learning_rate * db2

        # Update weights and biases with momentum
        W1 -= v_dW1
        b1 -= v_db1
        W2 -= v_dW2
        b2 -= v_db2

        if epoch % 10000 == 0:
            print(f"Epoch {epoch}, Loss: {loss}")

    return W1, b1, W2, b2

# Define function for predicting
def predict(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_propagation(X, W1, b1, W2, b2)
    return np.round(A2)


### Train and validate using 4-fold cross-validation

In [7]:
# Hyperparameters
learning_rate = 0.2
momentum_rate = 0.09
epochs = 150000
hidden_size = 2

# Perform 4-fold cross-validation
k = 4
fold_size = len(X_train) // k
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
best_weights = None

for fold in range(k):
    print(f'\n=== For Fold {fold+1} ===')
    # Split data into train and val sets
    X_fold_train = np.concatenate((X_train[:fold * fold_size], X_train[(fold + 1) * fold_size:]), axis=0)
    y_fold_train = np.concatenate((y_train[:fold * fold_size], y_train[(fold + 1) * fold_size:]), axis=0)
    X_fold_val = X_train[fold * fold_size:(fold + 1) * fold_size]
    y_fold_val = y_train[fold * fold_size:(fold + 1) * fold_size]

    # Train the model
    W1, b1, W2, b2 = train(X_fold_train, y_fold_train, hidden_size, learning_rate, momentum_rate, epochs)

    # Predictions
    y_pred = predict(X_fold_val, W1, b1, W2, b2)

    # Calculate TP, TN, FP, FN
    TP = np.sum((y_pred == 1) & (y_fold_val == 1))
    TN = np.sum((y_pred == 0) & (y_fold_val == 0))
    FP = np.sum((y_pred == 1) & (y_fold_val == 0))
    FN = np.sum((y_pred == 0) & (y_fold_val == 1))

    # Accuracy
    accuracy = (TP + TN) / len(y_fold_val)
    accuracy_scores.append(accuracy)

    # Precision
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    precision_scores.append(precision)

    # Recall
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    recall_scores.append(recall)

    # F1-score
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    f1_scores.append(f1)

    # Keep track of best weights and biases
    if best_weights is None or f1 == np.max(f1_scores):
        best_weights = {'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}

# cross validation performance metrics
print("\nAccuracy scores:", accuracy_scores)
print("Precision scores:", precision_scores)
print("Recall scores:", recall_scores)
print("F1 scores:", f1_scores)
print("\nBest Weights:", best_weights)


=== For Fold 1 ===
Epoch 0, Loss: 0.732873019625102
Epoch 10000, Loss: 0.16023590111678473
Epoch 20000, Loss: 0.07137059810935208
Epoch 30000, Loss: 0.04647842494961538
Epoch 40000, Loss: 0.0350398996588649
Epoch 50000, Loss: 0.026918800972093915
Epoch 60000, Loss: 0.020760438058904665
Epoch 70000, Loss: 0.01611380717466843
Epoch 80000, Loss: 0.012631929521740339
Epoch 90000, Loss: 0.010063772944728941
Epoch 100000, Loss: 0.008170165615542386
Epoch 110000, Loss: 0.006758690922960202
Epoch 120000, Loss: 0.005689312065235438
Epoch 130000, Loss: 0.004864317264594233
Epoch 140000, Loss: 0.004216305783863969

=== For Fold 2 ===
Epoch 0, Loss: 0.7127233865667766
Epoch 10000, Loss: 0.1486693071231661
Epoch 20000, Loss: 0.07883076731131508
Epoch 30000, Loss: 0.0555464557780053
Epoch 40000, Loss: 0.04248299121614781
Epoch 50000, Loss: 0.03160315061928437
Epoch 60000, Loss: 0.02469703917900714
Epoch 70000, Loss: 0.020427622498021176
Epoch 80000, Loss: 0.01767117933415973
Epoch 90000, Loss: 0.01

## Predictions on test data

In [8]:
# Use best weights for predictions on test data
W1 = best_weights['W1']
b1 = best_weights['b1']
W2 = best_weights['W2']
b2 = best_weights['b2']

y_pred_test = predict(X_test, W1, b1, W2, b2)

# Calculate performance metrics on test data
TP_test = np.sum((y_pred_test == 1) & (y_test == 1))
TN_test = np.sum((y_pred_test == 0) & (y_test == 0))
FP_test = np.sum((y_pred_test == 1) & (y_test == 0))
FN_test = np.sum((y_pred_test == 0) & (y_test == 1))

# Overall accuracy on test data
total_accuracy_test = (TP_test + TN_test) / len(y_test)

# Accuracy for class 0 (Non-Palindrome) on test data
class_0_accuracy_test = TN_test / (TN_test + FP_test) if (TN_test + FP_test) > 0 else 0

# Accuracy for class 1 (Palindrome) on test data
class_1_accuracy_test = TP_test / (TP_test + FN_test) if (TP_test + FN_test) > 0 else 0

# Print overall performance metrics on test data
print("\n=== Overall Performance on Test Data ===")
print("Total Accuracy:", total_accuracy_test)
print("Class 0 (Non-Palindrome) Accuracy:", class_0_accuracy_test)
print("Class 1 (Palindrome) Accuracy:", class_1_accuracy_test)


=== Overall Performance on Test Data ===
Total Accuracy: 0.9974747474747475
Class 0 (Non-Palindrome) Accuracy: 0.9945652173913043
Class 1 (Palindrome) Accuracy: 1.0


## Code Demo

In [9]:
# Change the array to provide your input
demoX = np.array([[1,0,0,1,1,1,1,0,0,1]])

W1 = best_weights['W1']
b1 = best_weights['b1']
W2 = best_weights['W2']
b2 = best_weights['b2']

_, _, _, A2 = forward_propagation(demoX, W1, b1, W2, b2)
print("Input:", demoX.squeeze())
print("Result:", (A2 > 0.5).squeeze())

Input: [1 0 0 1 1 1 1 0 0 1]
Result: True
